# RESELLER DIMENSION

In [1]:
import pandas as pd
from connection_script import connect_databases

In [2]:
db_op, db_etl = connect_databases()

## Extraction

In [3]:

store = pd.read_sql_query('SELECT * FROM [Sales].[Store]', db_op)
store_demographics = pd.read_sql_query('SELECT * FROM [Sales].[vStoreWithDemographics]', db_op)
person_phone = pd.read_sql_query('SELECT * FROM [Person].[PersonPhone]', db_op)
sales_person = pd.read_sql_query('SELECT * FROM [Sales].[SalesPerson]', db_op)
dim_geography = pd.read_sql_query('SELECT * FROM "DimGeography"', db_etl)
address = pd.read_sql_query('SELECT AddressID, AddressLine1, AddressLine2 FROM [Person].[Address]', db_op)
business_entity_address = pd.read_sql_query('SELECT * FROM [Person].[BusinessEntityAddress]', db_op)
sales_order_header = pd.read_sql_query('SELECT SalesOrderID,OrderDate,CustomerID FROM [Sales].[SalesOrderHeader] WHERE OnlineOrderFlag = 0', db_op)
customer = pd.read_sql_query('SELECT CustomerID,StoreID FROM [Sales].[Customer]', db_op)
store

,BusinessEntityID,Name,SalesPersonID,Demographics,rowguid,ModifiedDate
0,292,Next-Door Bike Store,279,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",A22517E3-848D-4EBE-B9D9-7437F3432304,2014-09-12 11:15:07.497
1,294,Professional Sales and Service,276,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",B50CA50B-C601-4A13-B07E-2C63862D71B4,2014-09-12 11:15:07.497
2,296,Riders Company,277,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",337C3688-1339-4E1A-A08A-B54B23566E49,2014-09-12 11:15:07.497
3,298,The Bike Mechanics,275,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",7894F278-F0C8-4D16-BD75-213FDBF13023,2014-09-12 11:15:07.497
4,300,Nationwide Supply,286,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",C3FC9705-A8C4-4F3A-9550-EB2FA4B7B64D,2014-09-12 11:15:07.497
...,...,...,...,...,...,...
696,1988,Retreat Inn,282,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",EA21EC81-1BFA-4A07-9B4D-73D9852AFCBF,2014-09-12 11:15:07.497
697,1990,Technical Parts Manufacturing,281,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",C8E3C4ED-8F58-4DB2-B600-E0CD11D9CFAD,2014-09-12 11:15:07.497
698,1992,Totes & Baskets Company,277,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",CE860B58-643C-4567-BFD8-06E97969CC67,2014-09-12 11:15:07.497
699,1994,World of Bikes,277,"<StoreSurvey xmlns=""http://schemas.microsoft.c...",0C10F2B6-A13A-440C-9C25-5B28D482872A,2014-09-12 11:15:07.497


## TRANSFORM

In [4]:
#Merging data frames
merged_df = store.merge(store_demographics, on='BusinessEntityID', how='inner')
merged_df = merged_df.rename(columns={'BusinessEntityID': 'StoreID'})
merged_df = merged_df.merge(person_phone, left_on='SalesPersonID', right_on='BusinessEntityID', how='inner')
merged_df = merged_df.merge(sales_person, left_on='SalesPersonID', right_on='BusinessEntityID', how='inner')
merged_df = merged_df.merge(dim_geography, left_on='TerritoryID', right_on='GeographyKey', how='inner')

#Remove duplicates keeping the latest address before merging
business_entity_address = business_entity_address.drop_duplicates(subset='BusinessEntityID', keep='last')

merged_df = merged_df.merge(business_entity_address, left_on='StoreID',right_on='BusinessEntityID', how='inner', suffixes=('_store', '_address'))
merged_df = merged_df.merge(address, left_on='AddressID', right_on='AddressID', how='inner')
print(merged_df.columns)
merged_df[['StoreID', 'Name_x', 'SalesPersonID', 'PhoneNumber', 'GeographyKey']]

Index(['StoreID', 'Name_x', 'SalesPersonID', 'Demographics', 'rowguid_x',
       'ModifiedDate_x', 'Name_y', 'AnnualSales', 'AnnualRevenue', 'BankName',
       'BusinessType', 'YearOpened', 'Specialty', 'SquareFeet', 'Brands',
       'Internet', 'NumberEmployees', 'BusinessEntityID_x', 'PhoneNumber',
       'PhoneNumberTypeID', 'ModifiedDate_y', 'BusinessEntityID_y',
       'TerritoryID', 'SalesQuota', 'Bonus', 'CommissionPct', 'SalesYTD',
       'SalesLastYear', 'rowguid_y', 'ModifiedDate_store', 'GeographyKey',
       'City', 'StateProvinceCode', 'StateProvinceName', 'CountryRegionCode',
       'EnglishCountryRegionName', 'PostalCode', 'SalesTerritoryID',
       'BusinessEntityID', 'AddressID', 'AddressTypeID', 'rowguid',
       'ModifiedDate_address', 'AddressLine1', 'AddressLine2'],
      dtype='object')


,StoreID,Name_x,SalesPersonID,PhoneNumber,GeographyKey
0,292,Next-Door Bike Store,279,664-555-0112,5
1,294,Professional Sales and Service,276,883-555-0116,4
2,296,Riders Company,277,517-555-0117,3
3,298,The Bike Mechanics,275,257-555-0154,2
4,300,Nationwide Supply,286,1 (11) 500 555-0190,9
...,...,...,...,...,...
696,1988,Retreat Inn,282,185-555-0169,6
697,1990,Technical Parts Manufacturing,281,330-555-0120,4
698,1992,Totes & Baskets Company,277,517-555-0117,3
699,1994,World of Bikes,277,517-555-0117,3


In [5]:
sales_order_header = sales_order_header.merge(customer[['CustomerID', 'StoreID']], on='CustomerID', how='left')
sales_order_header

,SalesOrderID,OrderDate,CustomerID,StoreID
0,43659,2011-05-31,29825,1046.0
1,43660,2011-05-31,29672,722.0
2,43661,2011-05-31,29734,852.0
3,43662,2011-05-31,29994,1418.0
4,43663,2011-05-31,29565,484.0
...,...,...,...,...
3801,71948,2014-05-01,29715,814.0
3802,71949,2014-05-01,29966,1356.0
3803,71950,2014-05-01,29678,734.0
3804,71951,2014-05-01,29642,654.0


In [6]:
def generateAlternateKey(ind):
    l = len(str(ind))
    zeros = 8 - l
    alternateKey = 'AW' + '0'*zeros + str(ind)
    return alternateKey


In [7]:
merged_df = merged_df.sort_values(by='StoreID')
merged_df = merged_df.reset_index(drop=False)
merged_df['index'] = merged_df['index'] + 1
merged_df.set_index('index', inplace=True)
merged_df['ResellerAlternateKey'] = merged_df.index.to_series().apply(generateAlternateKey)
merged_df[['StoreID', 'ResellerAlternateKey']]

,StoreID,ResellerAlternateKey
index,,
1,292,AW00000001
2,294,AW00000002
3,296,AW00000003
4,298,AW00000004
5,300,AW00000005
...,...,...
697,1988,AW00000697
698,1990,AW00000698
699,1992,AW00000699


In [8]:
# Convert OrderDate to datetime and extract the year
sales_order_header['OrderDate'] = pd.to_datetime(sales_order_header['OrderDate']).dt.year

# Find the first and last order year for each StoreID
order_years = sales_order_header.groupby('StoreID').agg(FirstOrderYear=('OrderDate', 'min'), LastOrderYear=('OrderDate', 'max')).reset_index()

# Merge with merged_df
merged_df = merged_df.merge(order_years, on='StoreID', how='left')

merged_df[['StoreID', 'ResellerAlternateKey', 'FirstOrderYear', 'LastOrderYear']]

,StoreID,ResellerAlternateKey,FirstOrderYear,LastOrderYear
0,292,AW00000001,2011.0,2013.0
1,294,AW00000002,2013.0,2014.0
2,296,AW00000003,2011.0,2014.0
3,298,AW00000004,2011.0,2014.0
4,300,AW00000005,2013.0,2014.0
...,...,...,...,...
696,1988,AW00000697,2012.0,2014.0
697,1990,AW00000698,2013.0,2014.0
698,1992,AW00000699,2011.0,2014.0
699,1994,AW00000700,2012.0,2014.0


In [9]:
# Column Selection
result = merged_df[['ResellerAlternateKey', 'GeographyKey', 'PhoneNumber', 'BusinessType', 'Name_x', 'NumberEmployees', 'FirstOrderYear', 'LastOrderYear','Specialty', 'AddressLine1', 'AddressLine2', 
           'AnnualSales', 'BankName', 'AnnualRevenue', 'YearOpened']]

#Renaming
# Create a copy of the DataFrame to avoid SettingWithCopyWarning
result = result.copy()

result = result.rename(columns={'PhoneNumber': 'Phone',
                        'Name_x': 'ResellerName',
                        'Specialty': 'ProductLine'})

result.index +=1

result

,ResellerAlternateKey,GeographyKey,Phone,BusinessType,ResellerName,NumberEmployees,FirstOrderYear,LastOrderYear,ProductLine,AddressLine1,AddressLine2,AnnualSales,BankName,AnnualRevenue,YearOpened
1,AW00000001,5,664-555-0112,BM,Next-Door Bike Store,13,2011.0,2013.0,Mountain,Mall Of Memphis,None,800000.0,United Security,80000.0,1996
2,AW00000002,4,883-555-0116,BM,Professional Sales and Service,14,2013.0,2014.0,Touring,57251 Serene Blvd,None,800000.0,International Bank,80000.0,1991
3,AW00000003,3,517-555-0117,BM,Riders Company,15,2011.0,2014.0,Road,Tanger Factory,None,800000.0,Primary Bank & Reserve,80000.0,1999
4,AW00000004,2,257-555-0154,BM,The Bike Mechanics,16,2011.0,2014.0,Mountain,Johnny Appleseed Shop.center,None,800000.0,International Security,80000.0,1994
5,AW00000005,9,1 (11) 500 555-0190,BM,Nationwide Supply,17,2013.0,2014.0,Touring,4250 Concord Road,None,800000.0,Guardian Bank,80000.0,1987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697,AW00000697,6,185-555-0169,BM,Retreat Inn,8,2012.0,2014.0,Road,Suite 2502 410 Albert Street,None,300000.0,Primary Bank & Reserve,30000.0,1982
698,AW00000698,4,330-555-0120,BM,Technical Parts Manufacturing,5,2013.0,2014.0,Touring,Ontario Mills,None,300000.0,International Security,30000.0,1976
699,AW00000699,3,517-555-0117,BM,Totes & Baskets Company,2,2011.0,2014.0,Road,72540 Blanco Rd.,None,300000.0,Guardian Bank,30000.0,1970
700,AW00000700,3,517-555-0117,BM,World of Bikes,17,2012.0,2014.0,Mountain,660 Lindbergh,None,800000.0,Primary Bank & Reserve,80000.0,1997


## LOAD

In [10]:
db_op, db_etl = connect_databases()
result.to_sql('DimReseller', db_etl, if_exists='replace', index_label='ResellerKey')

46

In [11]:
# Testing table creation
dim_reseller = pd.read_sql_query('SELECT * FROM "DimReseller"', db_etl)
dim_reseller

,ResellerKey,ResellerAlternateKey,GeographyKey,Phone,BusinessType,ResellerName,NumberEmployees,FirstOrderYear,LastOrderYear,ProductLine,AddressLine1,AddressLine2,AnnualSales,BankName,AnnualRevenue,YearOpened
0,1,AW00000001,5,664-555-0112,BM,Next-Door Bike Store,13,2011.0,2013.0,Mountain,Mall Of Memphis,None,800000.0,United Security,80000.0,1996
1,2,AW00000002,4,883-555-0116,BM,Professional Sales and Service,14,2013.0,2014.0,Touring,57251 Serene Blvd,None,800000.0,International Bank,80000.0,1991
2,3,AW00000003,3,517-555-0117,BM,Riders Company,15,2011.0,2014.0,Road,Tanger Factory,None,800000.0,Primary Bank & Reserve,80000.0,1999
3,4,AW00000004,2,257-555-0154,BM,The Bike Mechanics,16,2011.0,2014.0,Mountain,Johnny Appleseed Shop.center,None,800000.0,International Security,80000.0,1994
4,5,AW00000005,9,1 (11) 500 555-0190,BM,Nationwide Supply,17,2013.0,2014.0,Touring,4250 Concord Road,None,800000.0,Guardian Bank,80000.0,1987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
696,697,AW00000697,6,185-555-0169,BM,Retreat Inn,8,2012.0,2014.0,Road,Suite 2502 410 Albert Street,None,300000.0,Primary Bank & Reserve,30000.0,1982
697,698,AW00000698,4,330-555-0120,BM,Technical Parts Manufacturing,5,2013.0,2014.0,Touring,Ontario Mills,None,300000.0,International Security,30000.0,1976
698,699,AW00000699,3,517-555-0117,BM,Totes & Baskets Company,2,2011.0,2014.0,Road,72540 Blanco Rd.,None,300000.0,Guardian Bank,30000.0,1970
699,700,AW00000700,3,517-555-0117,BM,World of Bikes,17,2012.0,2014.0,Mountain,660 Lindbergh,None,800000.0,Primary Bank & Reserve,80000.0,1997
